# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [8]:
import findspark
findspark.init()
import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, Row, Window
from pyspark import SparkContext

sc = SparkContext(appName='SGA')
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 16:07:48,855 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [32]:
df = spark.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t')

df.createOrReplaceTempView('Clickstream')

spark.sql('''
    select distinct
        substring(t.route, 0, length(t.route) - 1) as route,
        count(*) over(partition by route) as count
    from (
        select
            substring_index(
                concat_ws(
                    '',
                    collect_list(
                        case event_type
                            when 'page'
                                then concat(event_page, '-')
                            when 'event'
                                then '' 
                            else 'CUSTOM_DELIMITER'
                        end
                    )
                ),
                'CUSTOM_DELIMITER',
                1
            ) as route
        from (select * from Clickstream order by user_id asc, session_id asc, timestamp asc, event_page asc)
        group by user_id, session_id
    ) t
    order by count desc
''').take(30)

[Row(route='main', count=8091),
 Row(route='main-archive', count=1094),
 Row(route='main-rabota', count=1039),
 Row(route='main-internet', count=879),
 Row(route='main-bonus', count=866),
 Row(route='main-news', count=760),
 Row(route='main-tariffs', count=669),
 Row(route='main-online', count=584),
 Row(route='main-vklad', count=512),
 Row(route='main-rabota-archive', count=167),
 Row(route='main-archive-rabota', count=166),
 Row(route='main-bonus-archive', count=139),
 Row(route='main-rabota-bonus', count=136),
 Row(route='main-bonus-rabota', count=134),
 Row(route='main-news-rabota', count=134),
 Row(route='main-archive-internet', count=130),
 Row(route='main-rabota-news', count=129),
 Row(route='main-internet-rabota', count=128),
 Row(route='main-archive-news', count=125),
 Row(route='main-internet-archive', count=123),
 Row(route='main-rabota-internet', count=123),
 Row(route='main-archive-bonus', count=117),
 Row(route='main-internet-bonus', count=114),
 Row(route='main-tariffs-i

In [33]:
def merge_value(acc, x):
    if x[0][0] == 'event' or not acc[-1][0]:
        return acc

    if x[0][0] != 'page' and x[0][0] != 'event':
        acc += [[False]]
        return acc

    if x[0][0] == 'page':
        acc += x

    return acc

def merge_combiners(acc, x):
    acc.extend(x)
    return acc

rdd = sc.parallelize(
    df.rdd \
    .sortBy(lambda x: (x.user_id, x.session_id, x.timestamp), ascending=True) \
    .map(lambda x: ((x.user_id, x.session_id), [[x.event_type, x.event_page]])) \
    .combineByKey(lambda x: x, merge_value, merge_combiners) \
    .map(lambda x: (x[0], [i for i in x[1] if len(i) == 2])) \
    .map(lambda x: '-'.join([i[1] for i in x[1]])) \
    .countByValue() \
    .items()
).sortBy(lambda x: x[1], ascending=False)

rdd.take(30)

[('main', 8089),
 ('main-archive', 1095),
 ('main-rabota', 1039),
 ('main-internet', 879),
 ('main-bonus', 864),
 ('main-news', 760),
 ('main-tariffs', 669),
 ('main-online', 585),
 ('main-vklad', 514),
 ('main-archive-rabota', 167),
 ('main-rabota-archive', 166),
 ('main-bonus-archive', 139),
 ('main-rabota-bonus', 137),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 133),
 ('main-archive-internet', 131),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 125),
 ('main-rabota-internet', 123),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 114),
 ('main-tariffs-internet', 113),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-internet-news', 102),
 ('main-tariffs-archive', 102),
 ('main-main', 94)]

In [34]:
from pyspark.sql.types import IntegerType

dt = df \
    .orderBy(['user_id', 'session_id', 'timestamp', 'event_page'], ascending=True) \
    .groupby(['user_id', 'session_id']) \
    .agg( \
        F.substring_index(
            F.concat_ws('', \
                F.collect_list( \
                    F.when(F.lower(F.trim(df['event_type'])) == 'page', F.concat(F.lower(F.trim(df['event_page'])), F.lit('-'))) \
                    .otherwise(
                        F.when(F.lower(F.trim(df['event_type'])) == 'event', F.lit('')) \
                        .otherwise('CUSTOM_DELIMITER') \
                    )) \
                ),
            'CUSTOM_DELIMITER', \
            1 \
            ).alias('route'), \
        ) \
    .withColumn('route_length', F.length('route') - 1) \
    .select(F.expr('substring(route, 0, route_length) as route')) \
    .groupBy('route') \
    .count() \
    .orderBy('count', ascending=False)

dt.head(30)

[Row(route='main', count=8091),
 Row(route='main-archive', count=1095),
 Row(route='main-rabota', count=1037),
 Row(route='main-internet', count=879),
 Row(route='main-bonus', count=865),
 Row(route='main-news', count=760),
 Row(route='main-tariffs', count=669),
 Row(route='main-online', count=584),
 Row(route='main-vklad', count=513),
 Row(route='main-archive-rabota', count=167),
 Row(route='main-rabota-archive', count=167),
 Row(route='main-bonus-archive', count=139),
 Row(route='main-rabota-bonus', count=136),
 Row(route='main-news-rabota', count=134),
 Row(route='main-bonus-rabota', count=134),
 Row(route='main-archive-internet', count=131),
 Row(route='main-rabota-news', count=129),
 Row(route='main-internet-rabota', count=128),
 Row(route='main-archive-news', count=125),
 Row(route='main-internet-archive', count=123),
 Row(route='main-rabota-internet', count=122),
 Row(route='main-archive-bonus', count=117),
 Row(route='main-internet-bonus', count=114),
 Row(route='main-tariffs-i

In [35]:
def save(rdd, path):
    result = [i[0] + '\t' + str(i[1]) for i in map(lambda x: list(x), rdd.take(30))]
    result.insert(0, 'route\tcount')
    result = '\n'.join(result)
    with open(path, 'w') as f:
        f.write(result)

save(rdd, "result_30_rows.txt")